In [1]:
import pandas as pd

In [19]:
df = pd.read_csv('ablations_raw_numbers.csv', na_values=[''])
df.head(10)

,attack,feature,norm,eps,unsafe_queries,total_queries
0,HSJA,NaN,l2,10.0,953,1731
1,HSJA,NaN,l2,20.0,339,701
2,HSJA,OPT grad estimation,l2,10.0,629,2528756
3,HSJA,OPT grad estimation,l2,20.0,212,869204
4,HSJA,OPT grad estimation and 2-stage search,l2,10.0,1227,52009
5,HSJA,OPT grad estimation and 2-stage search,l2,20.0,397,18778
6,OPT,NaN,l2,10.0,4975,9729
7,OPT,NaN,l2,20.0,1220,2449
8,OPT,line search,l2,10.0,735,3680863
9,OPT,line search,l2,20.0,260,1160345


In [29]:
import numpy as np


def get_improvements(attack: str, norm: str, eps: float) -> pd.DataFrame:
    attack_rows = df[(df['attack'] == attack) & (df['norm'] == norm) & (df['eps'] == eps)]
    original_row = attack_rows[attack_rows['feature'].isna()]
    assert len(original_row) == 1
    other_rows = attack_rows[attack_rows['feature'].notna()]
    unsafe_queries_improvements = other_rows['unsafe_queries'] / original_row['unsafe_queries'].item()
    total_queries_improvements = other_rows['total_queries'] / original_row['total_queries'].item()
    final_df = pd.DataFrame({
        'attack': attack,
        'feature': other_rows['feature'],
        'unsafe_queries_improvements': unsafe_queries_improvements,
        'total_queries_improvements': total_queries_improvements,
    })
    return final_df

In [30]:
get_improvements('OPT', 'l2', 10)

,attack,feature,unsafe_queries_improvements,total_queries_improvements
8,OPT,line search,0.147739,378.339295
10,OPT,2-stage line search,0.262915,4.951280


In [31]:
get_improvements('SignOPT', 'l2', 10)

,attack,feature,unsafe_queries_improvements,total_queries_improvements
14,SignOPT,optimal k,0.945681,0.926936
16,SignOPT,line search,0.794503,393.601684
18,SignOPT,line search and optimal k,0.551702,386.598990
20,SignOPT,2-stage line search,0.808246,4.986869
22,SignOPT,2-stage line search and optimal k,0.564136,4.793939


In [32]:
get_improvements('HSJA', 'l2', 10)

,attack,feature,unsafe_queries_improvements,total_queries_improvements
2,HSJA,OPT grad estimation,0.660021,1460.864240
4,HSJA,OPT grad estimation and 2-stage search,1.287513,30.045638


In [33]:
get_improvements('RayS', 'linf', 8)

,attack,feature,unsafe_queries_improvements,total_queries_improvements
28,RayS,line search,0.504098,3.421212
30,RayS,line search and early stopping,0.422131,3.424242
32,RayS,2-stage line search,0.655738,1.033333
34,RayS,2-stage line search and early stopping,0.565574,0.924242
